# Basketball Sports Betting Data Science Project

In [1]:
# needed libraries
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#function to retrieve playoffs results (Champion, Runner-up, Finals MVP, Points, Rebounds, Assists and Win Shares)

def get_champs():
    finals_url = "https://www.basketball-reference.com/playoffs/"
    html = urlopen(finals_url)
    soup = BeautifulSoup(html, features="lxml")
    headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
    
    #potential breakpoint if table schema changes
    headers.pop(5)

    rows = soup.findAll('tr')[2:]
    rows_data = [[td.getText() for td in rows[i].findAll('td')]
                    for i in range(len(rows))]
    
    nba_finals = pd.DataFrame(rows_data, columns = headers)
    nba_finals = nba_finals.dropna(how = "all")
    last_year = 2021
    years = []
    for i in range(0, nba_finals.shape[0]):
        years.append(last_year)
        last_year -=1
    nba_finals['Year'] = years
    return nba_finals

In [3]:
get_champs()

,Year,Lg,Champion,Runner-Up,Finals MVP,Points,Rebounds,Assists,Win Shares
0,2021,Milwaukee Bucks,Phoenix Suns,G. Antetokounmpo,,G. Antetokounmpo (634),G. Antetokounmpo (269),J. Holiday (199),G. Antetokounmpo (3.7)
1,2020,Los Angeles Lakers,Miami Heat,L. James,,A. Davis (582),L. James (226),L. James (184),A. Davis (4.5)
2,2019,Toronto Raptors,Golden State Warriors,K. Leonard,,K. Leonard (732),D. Green (223),D. Green (187),K. Leonard (4.9)
3,2018,Golden State Warriors,Cleveland Cavaliers,K. Durant,,L. James (748),D. Green (222),L. James (198),L. James (5.2)
4,2017,Golden State Warriors,Cleveland Cavaliers,K. Durant,,L. James (591),K. Love (191),L. James (141),L. James (4.3)
...,...,...,...,...,...,...,...,...,...
83,1938,None,None,None,None,None,None,None,None
84,1937,Minneapolis Lakers,Syracuse Nationals,,,G. Mikan (376),,J. Pollard (56),G. Mikan (3.7)
85,1936,Minneapolis Lakers,Washington Capitols,,,G. Mikan (303),,J. Pollard (39),G. Mikan (4.2)
86,1935,Baltimore Bullets,Philadelphia Warriors,,,J. Fulks (282),,H. Dallmar (37),C. Simmons (2.5)


#### TO DO:

Identify metrics like PER to calculate
Calculate ELO Ratings for each team

What do we need the model to perform?
1. Get historical games played and box scores
2. Use win/loss to calculate ELO rating and simulate to present day
    - Potentially this is impacted by average player PER for the team (coupled with other stats)
3. Simulate for daily games:
    - Probability vector for each team WIN/LOSS (for teams that are playing)
    - After all games, adjust ELO rating for the teams for the subsequent games
    - Adjust parameters of the model to increase accuracy
4. After test runs, review the accuracy of the model predictions and determine if model is deployable

## List of tables we need:
1. Teams (we need to map the teams name changes to the correct team)
2. Players
3. Seasons
4. Schedules



In [42]:
#Players - Master List of all players, active vs retired
def teams():
    teams_url = "https://www.basketball-reference.com/teams/"
    html = urlopen(teams_url)
    soup = BeautifulSoup(html, features="lxml")
    headers = [th.getText() for th in soup.findAll('tr')[0].findAll('th')]

    rows = soup.findAll('tr')[1:]
    rows_data = [[td.getText() for td in rows[i].findAll('td')]
                        for i in range(len(rows))]

    franchise_names = []

    for i in range(len(rows)):
        lst = [td.getText() for td in rows[i].findAll('a')]
        for string in lst:
            for char in string[0]:
                if char.isalpha():
                    alpha = True
            if alpha:    
                franchise_names.append(lst[0])
    
    return franchise_names


In [43]:
teams()

['Atlanta Hawks',
 'Boston Celtics',
 'Brooklyn Nets',
 'Charlotte Hornets',
 'Chicago Bulls',
 'Cleveland Cavaliers',
 'Dallas Mavericks',
 'Denver Nuggets',
 'Detroit Pistons',
 'Golden State Warriors',
 'Houston Rockets',
 'Indiana Pacers',
 'Los Angeles Clippers',
 'Los Angeles Lakers',
 'Memphis Grizzlies',
 'Miami Heat',
 'Milwaukee Bucks',
 'Minnesota Timberwolves',
 'New Orleans Pelicans',
 'New York Knicks',
 'Oklahoma City Thunder',
 'Orlando Magic',
 'Philadelphia 76ers',
 'Phoenix Suns',
 'Portland Trail Blazers',
 'Sacramento Kings',
 'San Antonio Spurs',
 'Toronto Raptors',
 'Utah Jazz',
 'Washington Wizards',
 'Anderson Packers',
 'Baltimore Bullets',
 'Chicago Stags',
 'Cleveland Rebels',
 'Denver Nuggets',
 'Detroit Falcons',
 'Indianapolis Jets',
 'Indianapolis Olympians',
 'Kentucky Colonels',
 'Memphis Sounds',
 'Pittsburgh Condors',
 'Pittsburgh Ironmen',
 'Providence Steam Rollers',
 'San Diego Sails',
 'Sheboygan Red Skins',
 'Spirits of St. Louis',
 'St. Louis B